<a href="https://colab.research.google.com/github/yukiharada1228/neural_network/blob/main/OpenCALM_7B%E3%81%AB%E3%82%88%E3%82%8B%E3%83%81%E3%83%A3%E3%83%83%E3%83%88%E3%83%9C%E3%83%83%E3%83%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# GPUの確認
import torch

is_cuda = torch.cuda.is_available()
device_name = ""
if is_cuda:
    device_name = torch.cuda.get_device_name()
print({"is_cuda": is_cuda, "device_name": device_name})

{'is_cuda': True, 'device_name': 'Tesla V100-SXM2-16GB'}


In [3]:
# ライブラリをインストール
%%capture
%pip install transformers peft accelerate bitsandbytes

In [4]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

# チャットボットを作成
class ChatBot:
    def __init__(self, max_length=128, k=40):
        hf_peft_repo = "yukiharada1228/open-calm-7b-instruct-lora-epoch1"
        peft_config = PeftConfig.from_pretrained(hf_peft_repo)
        model = AutoModelForCausalLM.from_pretrained(
            peft_config.base_model_name_or_path,
            return_dict=True,
            torch_dtype=torch.float16,
            device_map='auto'
            )
        self.model = PeftModel.from_pretrained(model, hf_peft_repo)
        self.tokenizer = AutoTokenizer.from_pretrained(peft_config.base_model_name_or_path)
        self.max_length = max_length
        self.k = k

    def chat(self):
        while "[exit]" not in (user_message := self._input()):

            prompt = self.generate_prompt(user_message)

            token_ids = self.tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

            output = ""
            for word in self.gradually_generate(token_ids):
                print(word, end='', flush=True)
                output += word
            print()

    def _input(self):
        s = input("> ").replace("\r", "").replace("\n", "")
        return s

    def generate_prompt(self, instruction, input=None, response=None):
      def add_escape(text):
        return text.replace('### Response', '###  Response')

      if input:
        prompt = f"""
    Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

    ### Instruction:
    {add_escape(instruction.strip())}

    ### Input:
    {add_escape(input.strip())}
    """.strip()
      else:
        prompt = f"""
    Below is an instruction that describes a task. Write a response that appropriately completes the request.

    ### Instruction:
    {add_escape(instruction.strip())}
    """.strip()

      if response:
        prompt += f"\n\n### Response:\n{add_escape(response.strip())}<|endoftext|>"
      else:
        prompt += f"\n\n### Response:\n"

      return prompt

    def gradually_generate(self, token_ids):
        token_ids = token_ids.to(self.model.device)
        for _ in range(self.max_length):
            with torch.no_grad():
                outputs = self.model(token_ids)

            logits = outputs.logits
            indices_to_remove = logits < torch.topk(logits, self.k)[0][..., -1, None]
            logits[indices_to_remove] = float('-inf')
            probs = torch.nn.functional.softmax(logits[..., -1, :], dim=-1)
            next_token_id = torch.multinomial(probs, num_samples=1)
            token_ids = torch.cat((token_ids, next_token_id), dim=-1)

            output_str = self.tokenizer.decode(next_token_id[0])

            yield output_str

            if "<|endoftext|>" in output_str:
                break

In [ ]:
# チャットを開始します
bot = ChatBot()

In [8]:
bot.chat()

> 機械学習と人工知能の違いについて教えてください
機械学習は、多くのコンピュータシミュレーションを行いながら特定のタスクを実行するのに役立つコンピュータシステムの能力を利用した研究技術であり、人工知能はそれとはかなり異なります。人工知能の主な研究目標は、人間から自律的に学習できるシステムを開発することです。したがって、人工知能は学習プロセスによって自律的に行動できるように設計されています。機械学習は、タスクの種類に依存し、多くのコンピュータ科学および機械学習研究プロジェクトを通して発展していきます。人工知能は、自律的でより倫理的であることが期待されているため、人間や動物のように振る舞い、より優れた認知を持つことができます。機械学習は、より高度な思考を持つ機械学習システムを開発することが目標であり、多くの研究が行われています。人工知能は、自律した知能を備えた自律システム
> [exit]
